In [1]:
from __future__ import division, print_function, absolute_import

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import glob
import numpy as np
import os

%pylab inline

image_width=20

pwd = os.getcwd()
tfrecord_file_path = pwd + "/AE_input_knock_ch1.tfrecord"
filename_queue = tf.train.string_input_producer(
                              tf.train.match_filenames_once(tfrecord_file_path),
                              shuffle=True, num_epochs=None)

reader = tf.TFRecordReader()
_, serialized_example = reader.read(filename_queue)
features = tf.parse_single_example(serialized_example,
                                       features={'data': tf.FixedLenFeature([], tf.string)}) 
img = tf.decode_raw(features['data'], tf.uint8)
img = tf.image.convert_image_dtype(img,tf.float32)
img = tf.reshape(img, [image_width * image_width])
input_batch = tf.train.shuffle_batch([img],batch_size=20,capacity=500,min_after_dequeue=250,num_threads=1)

# Training Parameters
learning_rate = 0.001
num_steps = 100000
display_step = 1000

# Network Parameters
num_hidden_1 = 20 # 1st layer num features
num_input = image_width * image_width

X = tf.placeholder("float", [None, num_input],name='input_x')


encoder_h1=tf.Variable(tf.random_normal([num_input, num_hidden_1]))
decoder_h1=tf.transpose(encoder_h1)

biases = {
    'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'decoder_b1': tf.Variable(tf.random_normal([num_input])),
}

# Building the encoder
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, encoder_h1),
                                   biases['encoder_b1']))
    return layer_1


# Building the decoder
def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, decoder_h1),
                                   biases['decoder_b1']))
    return layer_1

def cosine(a,b):
    a_norm = tf.sqrt(tf.reduce_sum(tf.pow(a,2)))
    b_norm = tf.sqrt(tf.reduce_sum(tf.pow(b,2)))
    a_b = tf.reduce_sum(tf.multiply(a,b))
    cosin = tf.divide(a_b,tf.multiply(a_norm,b_norm))
    return cosin

# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op
# Targets (Labels) are the input data.
y_true = X

# Define loss and optimizer, minimize the squared error

loss = tf.reduce_sum(tf.pow(y_true - y_pred, 2))
#loss = tf.reduce_sum(tf.subtract(tf.constant(1.0),cosine(y_true,y_pred)))
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss)

saver = tf.train.Saver()
tf.add_to_collection('output_y', decoder_op)
tf.add_to_collection('output_y', loss)

result = []

with tf.Session() as sess:
    
    tf.local_variables_initializer().run()
    tf.global_variables_initializer().run()
    coord = tf.train.Coordinator()
    thread = tf.train.start_queue_runners(sess=sess,coord=coord)
    
    for i in range(1,num_steps+1):
        _, l = sess.run([optimizer, loss],feed_dict={X: input_batch.eval()})
        result.append(l)
        if i % display_step == 0 or i == 1:
            print('Step %i: Minibatch Loss: %f' % (i, l))
            
    saver.save(sess,'saver/AE/AE')
    
    coord.request_stop()
    coord.join(thread)

Populating the interactive namespace from numpy and matplotlib
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Step 1: Minibatch Loss: 2014.564453
Step 1000: Minibatch Loss: 180.131714
Step 2000: Minibatch Loss: 106.401131
Step 3000: Minibatch Loss: 90.840088
Step 4000: Minibatch Loss: 78.056999
Step 5000: Minibatch Loss: 66.479889
Step 6000: Minibatch Loss: 61.702686
Step 7000: Minibatch Loss: 60.657715
Step 8000: Minibatch Loss: 47.265114
Step 9000: Minibatch Loss: 49.873505
Step 10000: Minibatch Loss: 43.602211
Step 11000: Minibatch Loss: 42.241547
Step 12000: Minibatch Loss: 34.072144
Step 13000: Minibatch Loss: 31.891312
Step 14000: Minibatch Loss: 27.506706
Step 15000: Minibatch Loss: 23.448593
Step 16000: Minibatch Loss: 20.891289
Step 17000: Minibatch Loss: 21.354143
Step 18000: M